In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,573 kB]
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,681

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the delayed flight data.
home_sales.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
home_sales.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [5]:
home_sales_2=home_sales
cols=["date_built","price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors", "waterfront", "view"]
for i in cols:
    home_sales_2 = home_sales_2.withColumn(i, home_sales[i].cast('float'))
    print(i)
    
home_sales_2.show()

date_built
price
bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|    2016.0|936923.0|     4.0|      3.0|     3167.0| 11733.0|   2.0|       1.0|76.0|
|7530a2d8-1ae3-451...|2021-06-13|    2013.0|379628.0|     2.0|      2.0|     2235.0| 14384.0|   1.0|       0.0|23.0|
|43de979c-0bf0-4c9...|2019-04-12|    2014.0|417866.0|     2.0|      2.0|     2127.0| 10575.0|   2.0|       0.0| 0.0|
|b672c137-b88c-48b...|2019-10-16|    2016.0|239895.0|     2.0|      2.0|     1631.0| 11149.0|   2.0|       0.0| 0.0|
|e0726d4d-d595-407...|2022-01-08|    2017.0|424418.0|     3.0|      2.0|     2249.0| 13878.0|   2.0| 

In [6]:
home_sales_2.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: float (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: float (nullable = true)
 |-- bathrooms: float (nullable = true)
 |-- sqft_living: float (nullable = true)
 |-- sqft_lot: float (nullable = true)
 |-- floors: float (nullable = true)
 |-- waterfront: float (nullable = true)
 |-- view: float (nullable = true)



In [7]:
# 2. Create a temporary view of the DataFrame.
home_sales_2.createOrReplaceTempView('home_sales')


In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""select bedrooms,
date_built,
round(AVG(price),2) as average_price
from home_sales 
where bedrooms=4
group by date_built,bedrooms
order by date_built asc
"""
).show()


+--------+----------+-------------+
|bedrooms|date_built|average_price|
+--------+----------+-------------+
|     4.0|    2010.0|    296800.75|
|     4.0|    2011.0|     302141.9|
|     4.0|    2012.0|    298233.42|
|     4.0|    2013.0|    299999.39|
|     4.0|    2014.0|    299073.89|
|     4.0|    2015.0|    307908.86|
|     4.0|    2016.0|    296050.24|
|     4.0|    2017.0|    296576.69|
+--------+----------+-------------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""select bedrooms,
bathrooms,
date_built,
round(AVG(price),2) as average_price
from home_sales 
where bedrooms=3 and bathrooms=3
group by bedrooms, bathrooms,date_built
order by date_built asc
"""
).show()


+--------+---------+----------+-------------+
|bedrooms|bathrooms|date_built|average_price|
+--------+---------+----------+-------------+
|     3.0|      3.0|    2010.0|    292859.62|
|     3.0|      3.0|    2011.0|    291117.47|
|     3.0|      3.0|    2012.0|    293683.19|
|     3.0|      3.0|    2013.0|    295962.27|
|     3.0|      3.0|    2014.0|    290852.27|
|     3.0|      3.0|    2015.0|     288770.3|
|     3.0|      3.0|    2016.0|    290555.07|
|     3.0|      3.0|    2017.0|    292676.79|
+--------+---------+----------+-------------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
start_time = time.time()

spark.sql("""select date_built,
round(AVG(price),2) as average_price
from home_sales 
where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000
group by date_built
order by date_built asc
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|    2010.0|    285010.22|
|    2011.0|    276553.81|
|    2012.0|    307539.97|
|    2013.0|    303676.79|
|    2014.0|    298264.72|
|    2015.0|    297609.97|
|    2016.0|     293965.1|
|    2017.0|    280317.58|
+----------+-------------+

--- 0.751441240310669 seconds ---


In [11]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select view,
round(AVG(price),2) as average_price
from home_sales 
where price>=350000
group by view
order by view + 0 asc
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 0.0|    403848.51|
| 1.0|    401044.25|
| 2.0|    397389.25|
| 3.0|     398867.6|
| 4.0|    399631.89|
| 5.0|    401471.82|
| 6.0|    395655.38|
| 7.0|    403005.77|
| 8.0|    398592.71|
| 9.0|    401393.34|
|10.0|    401868.43|
|11.0|    399548.12|
|12.0|    401501.32|
|13.0|    398917.98|
|14.0|    398570.03|
|15.0|     404673.3|
|16.0|    399586.53|
|17.0|    398474.49|
|18.0|    399332.91|
|19.0|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.7837388515472412 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select view,
round(AVG(price),2) as average_price
from home_sales 
where price>=350000
group by view
order by view + 0 asc
"""
).show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
| 0.0|    403848.51|
| 1.0|    401044.25|
| 2.0|    397389.25|
| 3.0|     398867.6|
| 4.0|    399631.89|
| 5.0|    401471.82|
| 6.0|    395655.38|
| 7.0|    403005.77|
| 8.0|    398592.71|
| 9.0|    401393.34|
|10.0|    401868.43|
|11.0|    399548.12|
|12.0|    401501.32|
|13.0|    398917.98|
|14.0|    398570.03|
|15.0|     404673.3|
|16.0|    399586.53|
|17.0|    398474.49|
|18.0|    399332.91|
|19.0|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.40529727935791016 seconds ---


In [15]:
#Using the cached data, run the query that filters out the average price per year built for houses with 3 bedrooms 3 bathrooms, 2 floors and are 
#over 2000 sqft and compare it to uncached runtime.
start_time = time.time()

spark.sql("""select date_built,
round(AVG(price),2) as average_price
from home_sales 
where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000
group by date_built
order by date_built asc
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|    2010.0|    285010.22|
|    2011.0|    276553.81|
|    2012.0|    307539.97|
|    2013.0|    303676.79|
|    2014.0|    298264.72|
|    2015.0|    297609.97|
|    2016.0|     293965.1|
|    2017.0|    280317.58|
+----------+-------------+

--- 0.4411585330963135 seconds ---


In [16]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_2.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partioned")

In [17]:
# 11. Read the parquet formatted data.
p_home_sales=spark.read.parquet("home_sales_partioned")

In [18]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView("P_home_Sales")

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""select view,
round(AVG(price),2) as average_price
from P_home_sales 
where price>=350000
group by view
order by view + 0 asc
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 0.0|    403848.51|
| 1.0|    401044.25|
| 2.0|    397389.25|
| 3.0|     398867.6|
| 4.0|    399631.89|
| 5.0|    401471.82|
| 6.0|    395655.38|
| 7.0|    403005.77|
| 8.0|    398592.71|
| 9.0|    401393.34|
|10.0|    401868.43|
|11.0|    399548.12|
|12.0|    401501.32|
|13.0|    398917.98|
|14.0|    398570.03|
|15.0|     404673.3|
|16.0|    399586.53|
|17.0|    398474.49|
|18.0|    399332.91|
|19.0|    398953.17|
+----+-------------+
only showing top 20 rows

--- 0.618605375289917 seconds ---


# This partitioning does not improve the speed because we are not making a query on date_built. However for the query below the run time improves slightly

In [23]:
# 13. Run the query that filters out the average price per year built for houses with 3 bedrooms 3 bathrooms, 2 floors and are 
#over 2000 sqft
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""select date_built,
round(AVG(price),2) as average_price
from P_home_sales 
where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000
group by date_built
order by date_built asc
"""
).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|    2010.0|    285010.22|
|    2011.0|    276553.81|
|    2012.0|    307539.97|
|    2013.0|    303676.79|
|    2014.0|    298264.72|
|    2015.0|    297609.97|
|    2016.0|     293965.1|
|    2017.0|    280317.58|
+----------+-------------+

--- 0.2813289165496826 seconds ---


In [24]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [25]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')


False